In [7]:
import cv2 as cv
import numpy as np
import time

import matplotlib.pyplot as plt
%matplotlib inline

In [8]:

#Loading Labels from Coco names
labels=[]
with open(r"C:\Users\Fernanda Januar\yolov3\data\coco.names") as f:
    for i in f:
        labels.append(i.strip())
f.close()
#print(len(labels))   #(80,)

#Loading YoloV3
cfg = r"C:\Users\Fernanda Januar\yolov3\cfg\yolov3.cfg"
weight = r"C:\Users\Fernanda Januar\yolov3\yolov3.weights"
network = cv.dnn.readNetFromDarknet(cfg, weight)

#Getting list names for all layers
layer_names = network.getLayerNames()
#print(len(layer_names))    #(254,)

#Getting only layer output names
output_layer_names = [layer_names[i-1] for i in network.getUnconnectedOutLayers()]
#print(output_layer_names)   #['yolo_82', 'yolo_94', 'yolo_106']

#Setting probability minimum to eliminate weak prediction
probability_minimum = 0.5

#Setting threshold to filtering weak bounding boxes with NMS
threshold = 0.3

#Generating colours for every detected object (bounding boxes) 
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')    #(80, 3)

print("loading yolov3... success")

loading yolov3... success


In [9]:
path = r"C:\Users\Fernanda Januar\traffic-cars.mp4"
video = cv.VideoCapture(path)
writer = None

f=0
t=0

while True:
    ret, frame = video.read()
    
    #Getting "BLOB" from frame
    blob = cv.dnn.blobFromImage(frame, scalefactor=1/255.0, size=(416,416), swapRB=True, crop=False)
    
    #Implementing Network yoloV3
    network.setInput(blob)
    
    start = time.time()
    output_network = network.forward(output_layer_names)
    end = time.time()
    
    f+=1
    t+=end-start
    print('Frame number {0} took {1:.5f} seconds'.format(f, end - start))
    
    #Getting bounding boxes, confidences, and class_numbers
    bounding_boxes = []
    confidences = []
    class_numbers = []
    h, w = frame.shape[:2]
    
    for output in output_network:
        for i in output:
            # Getting 80 classes' probabilities for current detected object
            scores = i[5:]
            
            #Getting index of the class with the maximum value of probability
            class_idx = np.argmax(scores)
            
            #Getting value of probability for defined class
            confidence_current = i[4]
            
            if confidence_current>probability_minimum:
                bounding_boxes_current = i[:4] * np.array([w,h,w,h])
                
                #Karna i[:4] hanya berisi -> [x_center, y_center, width, height], harus mencari [x_min, y_min]
                x_center, y_center, box_width, box_height = bounding_boxes_current
                x_min = int(x_center - (box_width/2))
                y_min = int(y_center - (box_height/2))
                
                # Adding results into prepared lists
                bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                class_numbers.append(class_idx)
                
    #Implementing non-maximum suppression  
    results = cv.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)
    
    #Drawing Bounding boxes and labels
    if len(results)>0:
        for i in results:
            #Getting coordinate bounding boxes
            x_min, y_min = bounding_boxes[i][:2]
            box_width, box_height = bounding_boxes[i][2:]
            
            #Drawing bounding boxes
            colour_box = colours[class_numbers[i]].tolist()
            cv.rectangle(frame, (x_min, y_min), (x_min+box_width, y_min+box_height), colour_box, 2)
            
            #Labels
            text = "{}".format(labels[class_numbers[i]])
            cv.putText(frame, text, (x_min-5, y_min-10), cv.FONT_HERSHEY_COMPLEX, 1.0, colour_box, 2)
            
    
    cv.imshow("Detection object", frame)
    if cv.waitKey(1) & 0xFF==ord("q"):
        break

video.release()
cv.destroyAllWindows()
            
# Printing final results
print('\nTotal number of frames', f)
print('Total amount of time {:.5f} seconds'.format(t))
print('FPS:', round((f / t), 1))



Frame number 1 took 0.47730 seconds
Frame number 2 took 0.37809 seconds
Frame number 3 took 0.35213 seconds
Frame number 4 took 0.33808 seconds
Frame number 5 took 0.39410 seconds
Frame number 6 took 0.37127 seconds
Frame number 7 took 0.36704 seconds
Frame number 8 took 0.33264 seconds
Frame number 9 took 0.37496 seconds
Frame number 10 took 0.33537 seconds
Frame number 11 took 0.33860 seconds
Frame number 12 took 0.38632 seconds
Frame number 13 took 0.35457 seconds
Frame number 14 took 0.39986 seconds
Frame number 15 took 0.33872 seconds
Frame number 16 took 0.36120 seconds
Frame number 17 took 0.35836 seconds
Frame number 18 took 0.34797 seconds
Frame number 19 took 0.32211 seconds
Frame number 20 took 0.32016 seconds
Frame number 21 took 0.35608 seconds
Frame number 22 took 0.37436 seconds
Frame number 23 took 0.34433 seconds
Frame number 24 took 0.35619 seconds
Frame number 25 took 0.35180 seconds
Frame number 26 took 0.31927 seconds
Frame number 27 took 0.36074 seconds
Frame numb